In [29]:
import evaluate
import numpy as np
import pandas as pd
from random import shuffle
import torch
import torch.utils.data
from transformers import AutoTokenizer, GPT2ForSequenceClassification, TrainingArguments, Trainer

In [27]:
with open("./toy_data/BigData.txt", "r") as f:
    XY = f.readlines()
print(XY[:5])
shuffle(XY)
train = XY[:int(len(XY)*0.8)]
test = XY[int(len(XY)*0.8):]
train_X = []
train_y = []
test_X = []
test_y = []

for line in train:
    x, y_ = line.split(",")
    train_X.append(x)
    train_y.append(y_)
for line in test:
    x, y_ = line.split(",")
    test_X.append(x)
    test_y.append(y_)

# train_X = pd.DataFrame(train_X)
# train_y = pd.DataFrame(train_y)
# test_X = pd.DataFrame(test_X)
# test_y = pd.DataFrame(test_y)




[' 1219529000  4946876000  80406000  8079652000  17000000  -20000000  1901000000  740000000  2081763000  350000000  228000000  258288000  15000000  165000000  782031000  6406000  -84000000  39713924000  14042733000  25377000000  2826000000  537235000  51000000  3000000  797000000  6000000  207000000  55000000  -23000000  3000000  42000000  3589050000  400000000  1150000000  79000000  450000000  148000000  104000000  521192000  1935192000  6153000000  5001000000  13619000000  1  2400000000  1580854677  6104102000  8257873000  1635000000  7763751000  1121328000  198000000  -43000000  9094000000  11422046000  21335693000  137000000  782000000  1970000000  2500000000  7700000000  50000000  -3000000  -848000000  2119000000  2345000000  4469000000  261000000  3990000000  5400000000  2109872000  206000000  1496000000  137000000  434000000  86000000  1400000000  1112000000  1293000000  64000000  184000000  4700000000  -194000000  222000000  -23000000  19000000  240000000  30000000  3300000000 

In [31]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)

def tokenize_function(text):
    return tokenizer(text, padding=True, truncation=True, return_tensors="pt")

tokenizer.pad_token = tokenizer.eos_token

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

tokenized_train_X = [tokenize_function(sentence) for sentence in train_X]
tokenized_test_X = [tokenize_function(sentence) for sentence in test_X]

print(tokenized_train_X[:5])

train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_X), torch.tensor(train_y))
eval_dataset = torch.utils.data.TensorDataset(torch.tensor(test_X), torch.tensor(test_y))


training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

inputs = tokenizer(train_X, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()

labels = torch.tensor(train_y)
loss = model(**inputs, labels=labels).loss


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'input_ids': tensor([[  604, 16945, 10535,   220,   718, 26200, 10535,   220,  5014,    19,
         10535,   220, 15696,  1558, 10535,   220, 15709,   220, 15709,   220,
          1679,  4051, 10535,   220,   532, 45191, 24598,   220,   532,  3829,
          2791, 10535,   220, 15709,   220, 15709,   220,   642,  2167, 20483,
           220, 15709,   220, 15709,   220, 15709,   220,   642, 25600,   830,
           220, 15709,   220, 37517,  3865, 10535,   220, 32996,  2231, 10535,
           220, 15709,   220, 15709,   220, 15709,   220, 15709,   220, 15709,
           220, 15709,   220, 15709,   220, 15709,   220, 15709,   220, 15709,
           220, 15709,   220, 15709,   220, 15709,   220, 15709,   220, 15709,
           220, 15709,   220, 15709,   220, 15709,   220, 15709,   220,  2319,
          2780, 10535,   220, 15709,   220, 15709,   220, 15709,   220, 15709,
           220, 15709,   220,  1987,  8269,   220,  2757,  3270, 22913,  1821,
           220, 15709,   220, 15709, 

ValueError: too many dimensions 'str'

torch.Size([160, 1024])
